In [1]:
import numpy as np 
import pandas as pd

meta = pd.read_csv('./data/Meta/meta_info.csv')
meta.head()

,patient_id,original_image,mask_image,malignancy,is_cancer,is_clean
0,1,0001_img_slice086,0001_mask_slice086,5,True,False
1,1,0001_img_slice087,0001_mask_slice087,5,True,False
2,1,0001_img_slice088,0001_mask_slice088,5,True,False
3,1,0001_img_slice089,0001_mask_slice089,5,True,False
4,1,0001_img_slice090,0001_mask_slice090,5,True,False


In [9]:
import cv2

max_area = 0
bbox_list = []

for i in range (len(meta.index)):

    if meta.iloc[i]['is_clean'] == False:
        mask_array = np.load('./data/Mask/LIDC-IDRI-' + meta.iloc[i]['mask_image'][:4] + '/' + meta.iloc[i]['mask_image'] + '.npy')
        # Convert the mask to a binary image
        binary_mask = np.uint8(mask_array) * 255
        contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        bounding_boxes = []
        for contour in contours:
            x,y,w,h = cv2.boundingRect(contour)
            if w*h<20:
                continue
            bounding_boxes.append([x,y,x+w,y+h])
        bbox_list.append(bounding_boxes)
        
    else :
        mask_array = np.load('./data/Clean/Mask/LIDC-IDRI-' + meta.iloc[i]['mask_image'][:4] + '/' + meta.iloc[i]['mask_image'] + '.npy')
        bbox_list.append([[0,0,0,0]])

In [5]:
import pickle

with open("./data/bbox.pkl", "wb") as f:
    pickle.dump(bbox_list, f)